In [42]:
import pandas as pd
from numpy import int64
import requests
import json
from pathlib import Path
#from pandasql import sqldf

# Cleaning Data to Construct the DataSet Needed for the Visualizing CO2 Emissions Project<br>
-- Import software packages and read in CSVs. Although these data sets are technically available via API call, I emailed with both entities, and they requested since the sets are relatively small that I use CSVs. <br>
-- On the Per Capita CO2 csv I use a list comprehension to rename "Entity" to "Country" at the head of series one, and "Annual CO2 emissions (per capita)" in series 3 for the sake of clarity. I also filter out the country code for now. <br>
-- I repeat the process for the co2 emissions totals as well.

In [43]:
capita = pd.read_csv('/Users/lydia/Projects/PerCapitaCO2Emissions_data_visualization/data/co-emissions-per-capita.csv')

In [44]:
capita.columns=['Country' if x== 'Entity' else 'Per-Capita CO₂ emissions' if x== 'Annual CO₂ emissions' else x for x in capita.columns]  

In [45]:
capita = capita.filter(['Country', 'Year', 'Annual CO₂ emissions (per capita)'], axis=1)

In [46]:
capita.head()

,Country,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,1949,0.001992
1,Afghanistan,1950,0.011266
2,Afghanistan,1951,0.012098
3,Afghanistan,1952,0.011946
4,Afghanistan,1953,0.013685


In [47]:
total_country = pd.read_csv('/Users/lydia/Projects/PerCapitaCO2Emissions_data_visualization/data/annual-co2-emissions-per-country.csv')

In [48]:
total_country.columns=['Country' if x== 'Entity' else x for x in total_country.columns]  

In [49]:
total_country = total_country.filter(['Country', 'Year', 'Annual CO₂ emissions'], axis=1)

In [50]:
total_country.head()

,Country,Year,Annual CO₂ emissions
0,Afghanistan,1949,14656.0
1,Afghanistan,1950,84272.0
2,Afghanistan,1951,91600.0
3,Afghanistan,1952,91600.0
4,Afghanistan,1953,106256.0


-- Next I combine the two sets. The illustrations will be based on the per capita numbers, but I will need the totals for context. 

In [51]:
co2df = pd.merge(capita, total_country, how = 'left', left_on=['Country', 'Year'], right_on = ['Country', 'Year'])

In [52]:
co2df.head()

,Country,Year,Annual CO₂ emissions (per capita),Annual CO₂ emissions
0,Afghanistan,1949,0.001992,14656.0
1,Afghanistan,1950,0.011266,84272.0
2,Afghanistan,1951,0.012098,91600.0
3,Afghanistan,1952,0.011946,91600.0
4,Afghanistan,1953,0.013685,106256.0


-- The World Bank datasets have a few contextual fields at the top of the frame that confuse matters. As I read the CSV in, I choose not to bring those rows in.<br>
--To start working on the set I changed to "Country Name" column to "Country" for continuity later.

In [53]:
gdp = pd.read_csv('/Users/lydia/Projects/PerCapitaCO2Emissions_data_visualization/data/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_26/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_26.csv', skiprows = range(0,3))

In [54]:
gdp.rename(columns={'Country Name':'Country'}, inplace=True)

-- As you can see below, the makeup of the World Bank tables is such that there are singular countries, and many columns for years. Some of the columns are not useful for these purposes, so I don't want to see them. As there are many useful columns and a few less so, I use a list comprehension to iterate and check the column names against a list of unwanted columns, and ultimately return a df of only the desired information. 

In [55]:
gdp.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.558906e+09,3.103184e+09,3.544708e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,1.847810e+10,1.936631e+10,2.050647e+10,2.224273e+10,2.429433e+10,2.661956e+10,...,9.325135e+11,8.900514e+11,1.028394e+12,1.012521e+12,1.006191e+12,9.288802e+11,1.086531e+12,1.185138e+12,NaN,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426650e+10,NaN,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.041165e+10,1.113592e+10,1.195171e+10,1.268581e+10,1.384900e+10,1.487476e+10,...,7.692632e+11,6.921149e+11,6.856303e+11,7.681582e+11,8.234056e+11,7.869624e+11,8.449275e+11,8.753937e+11,NaN,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.049642e+10,5.276162e+10,7.369016e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.067828e+11,NaN,NaN


In [56]:
gdp.drop(gdp.columns[gdp.columns.str.contains('Unnamed: 68',case = False)], axis = 1, inplace = True)

In [57]:
undesired_columns = ['Country Code', 'Indicator Name', 'Indicator Code']
gdp = gdp[[i for i in gdp.columns if i not in undesired_columns]]

-- Next I pivot the table to have 3 columns instead of 60-some, and make them match to the OWID format. 

In [58]:
gdp = pd.melt(gdp, id_vars=['Country'],var_name='Year', value_name='GDP')

In [59]:
gdp = gdp.sort_values(by=['Country', 'Year'])

In [60]:
gdp.head()

,Country,Year,GDP
2,Afghanistan,1960,5.377778e+08
268,Afghanistan,1961,5.488889e+08
534,Afghanistan,1962,5.466667e+08
800,Afghanistan,1963,7.511112e+08
1066,Afghanistan,1964,8.000000e+08


-- I repeat the process for population

In [61]:
pop = pd.read_csv('/Users/lydia/Projects/PerCapitaCO2Emissions_data_visualization/data/API_SP.POP.TOTL_DS2_en_csv_v2_84031/API_SP.POP.TOTL_DS2_en_csv_v2_84031.csv', skiprows = range(0,3))

In [62]:
pop.rename(columns={'Country Name':'Country'}, inplace=True)

In [63]:
undesired_columns = ['Country Code', 'Indicator Name', 'Indicator Code']
pop = pop[[i for i in pop.columns if i not in undesired_columns]]

In [64]:
pop = pd.melt(pop, id_vars=['Country'],var_name='Year', value_name='Population')

In [65]:
pop = pop.sort_values(by=['Country', 'Year'])

In [66]:
pop.head()

,Country,Year,Population
2,Afghanistan,1960,8622466.0
268,Afghanistan,1961,8790140.0
534,Afghanistan,1962,8969047.0
800,Afghanistan,1963,9157465.0
1066,Afghanistan,1964,9355514.0


-- Next I merge the 2 World Bank data sets, now showing only the data I need, and in the format of the OWID sets.<br>
-- After that I intend to merge the 2 data sets into one, but the datetypes do not all match, so I need to do more cleaning. 

In [67]:
wbdf = pd.merge(gdp, pop, how = 'left', left_on=['Country', 'Year'], right_on = ['Country', 'Year'])

In [68]:
wbdf.dtypes

Country        object
Year           object
GDP           float64
Population    float64
dtype: object

In [69]:
co2df.dtypes

Country                               object
Year                                   int64
Annual CO₂ emissions (per capita)    float64
Annual CO₂ emissions                 float64
dtype: object

In [70]:
wbdf['Year'] = wbdf['Year'].fillna(0)

In [71]:
wbdf['Year'] = wbdf['Year'].astype(int64)

In [72]:
co2df= pd.merge(co2df, wbdf, how = 'left', left_on=['Country', 'Year'], right_on = ['Country', 'Year'])

In [73]:
co2df['GDP'] = co2df['GDP'].fillna(0)
co2df['Population'] = co2df['Population'].fillna(0)

For the purposes of graphing I need to establish a new DataFrame with only the top 20 emissions per capita countries. Then I need numbered indexes for gdp order, population order, and to reindex by the greatest annual co2 emissions per capita.

In [74]:
co2df = co2df[co2df['Year'] == 2022]
co2df = co2df[co2df['Population'] > 5000000]
co2df = co2df[co2df['Country'] != "North America"]

In [75]:
co2df = co2df.sort_values(by='Annual CO₂ emissions (per capita)', ascending = False)

In [76]:
top_twenty = pd.DataFrame(co2df.head(20))

In [ ]:
top_twenty = top_twenty.reset_index(drop=True)

In [ ]:
top_twenty_list = top_twenty['Country'].tolist()

In [ ]:
top_twenty= top_twenty.sort_values(by='GDP')

In [ ]:
top_twenty['GDP_Order'] = range(1, len(top_twenty)+1)

In [ ]:
top_twenty= top_twenty.sort_values(by='Population')

In [ ]:
top_twenty['Population_Order'] = range(1, len(top_twenty)+1)

The top_twenty dataframe is what I need to start graphing the main page of my visualization.

In [ ]:
top_twenty.head(20)

From the top_twenty_list I will need to generate views for twenty years of each of the twenty countries. 

In [ ]:
c_filter = capita[['Country', 'Year', Annual CO2 emissions (per capita)]].loc[capita['Year'] >= 2003]
c_filter.replace(' ', '_', regex=True)
c_filter = c_filter.sort_values(by='Year')

In [ ]:
c_filter.head()

In [ ]:
print(top_twenty_list)

In [ ]:
radar_United_Arab_Emirates = c_filter[c_filter['Country'] == "United Arab Emirates"]
radar_United_Arab_Emirates = pd.DataFrame(radar_United_Arab_Emirates)
radar_United_Arab_Emirates.head(20)